# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import warnings

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
# Read to csv and preview
output_data_file = "../WeatherPy/output_data/clean_weather.csv"
vacation_df = pd.read_csv(output_data_file)
vacation_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yangi Marg`ilon,40.4272,71.7189,69.22,37,4,6.91,UZ,2021-06-27 00:51:15
1,1,Hambantota,6.1241,81.1185,75.54,100,20,5.75,LK,2021-06-27 00:51:16
2,2,Fort-Shevchenko,44.5086,50.2630,85.33,39,98,14.27,KZ,2021-06-27 00:51:16
3,3,Bintulu,3.1667,113.0333,75.61,100,40,3.44,MY,2021-06-27 00:51:17
4,4,Vaini,-21.2000,-175.2000,75.36,73,20,13.80,TO,2021-06-27 00:51:17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = vacation_df[["Latitude", "Longitude"]]
humdiity = vacation_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humdiity, dissipating=False, max_intensity=100, point_radius=1)

# Add Layer
fig.add_layer(heat_layer)

# Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
ideal_df = vacation_df.loc[(vacation_df["Max Temperature"]<80) & (vacation_df["Max Temperature"]>70) & (vacation_df["Wind Speed"]<10) & (vacation_df["Cloudiness"]==0)]
ideal_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,Manicoré,-5.8092,-61.3003,76.95,94,0,1.14,BR,2021-06-27 00:46:50
84,84,Trairi,-3.2778,-39.2689,74.50,85,0,3.36,BR,2021-06-27 00:47:17
249,249,Trindade,-16.6494,-49.4889,73.15,43,0,6.51,BR,2021-06-27 00:53:32
336,336,Murgab,37.4966,61.9714,71.47,43,0,4.61,TM,2021-06-27 00:54:12
360,360,Abu Samrah,35.3029,37.1841,75.25,53,0,6.29,SY,2021-06-27 00:54:24
410,410,Alexandria,31.2156,29.9553,76.96,88,0,9.22,EG,2021-06-27 00:54:05
501,501,Olinda,-8.0089,-34.8553,71.33,73,0,9.22,BR,2021-06-27 00:55:31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Create new hotel dataframe with new column for Hotel Name
hotel_df = ideal_df.loc[:,["City", "Latitude", "Longitude", "Max Temperature", "Humidity", "Cloudiness", "Wind Speed", "Country"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

# Set parameters to search for hotels w/in 5000 meters
params = {"radius": 5000, "types": "lodging", "key": g_key}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Change location eah iteation while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the serach term: 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make a reqeust
    response = requests.get(base_url, params=params)
    # print(response.url)

    # Convert to json
    json_response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = json_response[0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


# My Google API key is not working. I created a new one that I left unrestricted just to make sure. I printed the URL and it says I'm not authorized to use that API (which matches the file I've saved) even though I made sure it wasn't restricted. Someone else in class also had this issue so I don't know if it's Google or us.

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [16]:
# Show hotel_df even though hotel names were not added
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
62,Manicoré,-5.8092,-61.3003,76.95,94,0,1.14,BR,
84,Trairi,-3.2778,-39.2689,74.50,85,0,3.36,BR,
249,Trindade,-16.6494,-49.4889,73.15,43,0,6.51,BR,
336,Murgab,37.4966,61.9714,71.47,43,0,4.61,TM,
360,Abu Samrah,35.3029,37.1841,75.25,53,0,6.29,SY,
410,Alexandria,31.2156,29.9553,76.96,88,0,9.22,EG,
501,Olinda,-8.0089,-34.8553,71.33,73,0,9.22,BR,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

# I've checked and double checked my API. I even created a new one that was unrestricted and then waited several hours to make sure it was activated. Still does not work. Someone else had problems as well, not sure if it's us or Google.

Figure(layout=FigureLayout(height='420px'))